# 4. Anvendelse af ANN til billedgenkendelse

I denne klasse vil vi kombinere den viden, vi hidtil har opnået, både om billeder og om klassifikation med neurale netværk (ANN), og bruge det til mere praktiske og sjove anvendelser. Lad os lære, hvordan vi bruger ANN til at klassificere billeder.

Vi vil bruge et [datasæt](https://www.kaggle.com/datasets/jorgebuenoperez/datacleaningglassesnoglasses) fra [Kaggle](https://www.kaggle.com), bestående af ansigter af mere end 4000 mennesker. Nogle af dem bærer briller, nogle gør ikke. Lad os udvikle en ANN-model, der vil genkende dette.

Det første skridt er at få lidt mere viden om ANN og lære, hvilken type neurale netværk der specifikt er god til billeder. Men før vi gør dette, lad os genskabe nogle af de funktioner, vi har oprettet sidst i den forrige klasse:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def table(reference, predicted):
    """ computes contingency table for predicted and reference class label indices """
    indices = reference.unique()
    n = len(indices)
    ct = np.zeros((n, n))
    for i in range(n):
        ni = sum(reference == indices[i])
        for j in range(n):
            ct[i, j] = sum((reference == indices[i]) & (predicted == indices[j])) / ni

    return ct

def ct_heatmap(ct, classes):
    """ shows heatmap for the table """
    plt.imshow(ct, clim = [0, 1])
    plt.colorbar()

    n = len(classes)
    plt.gca().set_xticks(range(n), classes)
    plt.gca().set_yticks(range(n), classes)
    for i in range(n):
        for j in range(n):
            plt.text(i, j, round(ct[j, i], 3), color = "white" if ct[j, i] < 0.5 else "black")

Som du måske har bemærket, kopierede vi ikke metoderne `train()` og `predict()` her, fordi vi vil lære en ny måde at træne på og vil skrive nye funktioner til det.

Lad os nu lære lidt teori.

## Konvolutionelle neurale netværk

Konvolutionelle neurale netværk (CNN'er) er en type af Kunstige Neurale Netværk (ANN'er) designet specifikt til billedklassificeringsopgaver. CNN'er er inspireret af det menneskelige visuelle system og er særligt effektive til at behandle gitter-lignende data, såsom billeder (indeholdende pixels).

Hvorfor ikke bruge de simple ANN'er, vi lærte om i den forrige klasse? Fordi de kræver features som input - nogle målinger, der er relevante for klassifikationen. I tilfældet med Iris-data brugte vi geometriske målinger af blomsterne, fordi de faktisk er forskellige for forskellige Iris-arter.

Når det kommer til billeder, har vi ikke features, vi har billeder. Og det, vi har brug for, er at tage billederne og beregne relevante features, der vil blive brugt til klassifikationen. Dette er præcis, hvordan CNN fungerer.

Lad os først kigge på CNN's arkitekturens hovedstruktur og dens vigtigste komponenter. En generel illustration af et konvolutionelt neuralt netværk er vist nedenfor:

<img src="./images/CNN-Architecture.png" style="width:800px; height:350px;"/>

Dermed består CNN's arkitektur af to blokke. Den første blok er nødvendig for at skabe features fra billeder, og den består normalt af følgende komponenter:

- **Inputlag:** netværket tager et billede som input, så inputtet er 2D (til gråtonebilleder) eller 3D (til farvebilleder) ikke bare en vektor af værdier som i det netværk, vi brugte i den forrige klasse.

- **Konvoluteringslag:** disse lag er de centrale byggeblokke i CNN'er. De anvender forskellige filtre (som dem vi lærte om i den første klasse) på inputbillederne for at afsløre forskellige mønstre og features, såsom kanter, konturer, lyse og mørke pletter, teksturer eller mere komplekse strukturer.

- **Aktiveringsfunktion:** ligesom i simple ANN'er kan aktiveringsfunktioner også anvendes på outputtet fra de konvolutterende lag. De introducerer ikke-linearitet, hvilket gør det muligt for netværket at lære mere komplekse forhold i dataene.

- **Poolinglag:** poolinglag er nødvendige for at reducere størrelsen på features skabt af konvolutterende lag. De samler på en måde features og beholder kun de vigtigste oplysninger.

Disse tre typer lag har en særlig egenskab - de tager billeder som input (ikke nødvendigvis billeder, det kan være enhver 3D-array, som har bredde, højde og antal kanaler eller skiver) og producerer et billede som output.

Den næste blok af lag arbejder bare med numeriske tabellerede data, som vi brugte til Iris-klassifikationen i den forrige klasse. Dette betyder, at outputtet fra den foregående blok skal omformes fra 3D til en simpel vektor af tal for at fortsætte. Dette kan gøres ved en speciel operation kaldet *fladning* (eller *udfoldning*). Derefter sendes informationen videre til de næste lag: 

- **Fuldt forbundne lag:** de er helt de samme som dem, vi brugte i den forrige klasse til Iris-klassifikationen. De tager input som en sæt af tal og producerer også output som et tal. Normalt suppleret med en aktiveringsfunktion.

- **Outputlag:** igen, det samme som vi brugte før - de indsamler output fra de fuldt forbundne lag og indsnævrer dem til et eller flere endelige output.

Man kan sige, at konvolutions- og pooling-lag er nødvendige for at konstruere forskellige features, der repræsenterer forskellige karakteristika ved billedet (f.eks. tilstedeværelsen af lodrette, vandrette eller diagonale linjer, cirkler osv.). Mens de fuldt forbundne lag og output-laget bruger disse egenskaber til at foretage klassificeringen, ligesom i tilfældet med Iris-data.

Dette er en simpel datastrøm i et typisk CNN:

$Input→Konvolution→Pooling→Udfladning→Fuldt forbundne lag→Output$

Lad os nu lære mere om de nye lag.

### Konvolution

Hvad er en *konvolution*? Du brugte allerede konvolution i den første klasse, da du prøvede forskellige filtre til billeder. Operationen med at beregne intensiteten af de endelige pixels baseret på den lineær kombination af deres naboer og vægtene af filteret kaldes en *konvolution*.

I tilfælde af CNN er konvolution en måde at beregne forskellige features for billedpixels. Lad os huske, hvordan billeder kan repræsenteres som en matrix med tal:

<img src="./images/Original Image-Pixels.png" style="width:800px; height:350px;"/>

I dette tilfælde bruger vi værdierne 0 (for sort) og 1 (for hvid) bare for enkelhedens skyld i stedet for 0 og 255.

For at anvende en konvolution på dette simple billede skal vi definere et *filter* (også kendt som en *kerne*) - en lille gitter af tal, som vil glide fra en pixel til en anden. Disse tal er vægte, som bruges til at beregne en vægtet sum af de originale pixelintensiteter. Resultatet af denne beregning er den nye værdi, som vi kalder en *feature*.

Ved at anvende denne operation på hver pixel i det originale (input) billede skaber vi et *feature map*. Du kan tænke på feature map'et som et andet billede, baseret på det oprindelige. Her er et eksempel:

<img src="./images/Filters-General.png" style="width:800px; height:1200px;"/>

Og her er en interaktiv illustration af denne proces også for et 3x3 filter :

<img src="./images/Image-Kernel-Filter.gif" style="width:800px; height:300px;"/>

Det er præcis hvad konvolutionslaget gør - skaber feature map.

På dette trin anbefales det at åbne den tredje ark i [Excel regnearket](../mlcourse.xlsm) og eksperimentere med filtrerings/konvolutions-eksemplet for at genopfriske, hvordan det fungerer. Prøv f.eks. at anvende filteret fra illustrationen ovenfor.

Hvad der gør denne procedure anderledes i tilfælde af CNN, er, at vi ikke definerer vægtene af filtrene, kun deres størrelse. Vægtene oprettes automatisk under læringsprocessen. Med andre ord genererer CNN automatisk features, som er bedst egnet til klassificering eller andre opgaver. Det lærer bogstaveligt talt fra dataene, vi definerer bare antallet af lagene og størrelsen af kernerne.

Når konvolutionen er udført, anvender netværket en aktiveringsfunktion på feature maps, ligesom vi også gjorde for vores simple model til Iris-klassificering. For eksempel, hvis ReLU bruges som aktiveringsfunktion, vil den erstatte alle negative værdier i feature map'et med 0, som vist nedenfor.

<img src="./images/Filter-Activation2.png" style="width:800px; height:250px;"/>

#### Skridt og fyldning

Udover filterstørrelsen har det konvolutinelle lag to andre vigtige parametre — *skridt* og *fyldning*.

**Skridt** er et trin filteret bevæger sig inde i billedet. Hvis skridtet er lig med en, flytter det simpelthen filtervinduet fra en pixel til en anden. Hvis skridtet er større, hopper det med et større skridt. 

Her er en illustration fra Wikimedia, der viser en konvolution med et skridt lig med 2, så den behandler hver anden række (2 og 4 i dette tilfælde) og hver anden kolonne (2 og 4 også):

<img src="images/strides.gif">

**Fyldning** er nødvendig for at behandle alle pixels og undgå størrelsesreduktion efter filtrering. Som du kan se i eksemplet ovenfor, kan et 3x3 filter ikke starte fra række 1 og kolonne 1. Grænsepixelerne behandles ikke, fordi en del af filteret i dette tilfælde vil være uden for billedet. For at undgå denne situation kan man tilføje fyldning (padding) omkring de originale billeder (normalt udfyldt med nuller).

Her er en illustration fra Wikimedia, hvor konvolution stadig fungerer med skridt (stride) = 2, men denne gang er der tilføjet fyldning (padding), så filteret tager række 1, 3 og 5 og de samme kolonner:

<img src="images/padding-strides.gif">

Forsøg at implementere fyldning (padding) for filtreringseksemplet i Excel-regnearket.

### Pooling

Efter aktiveringsfunktionen er anvendt på konvolutionsresultaterne, fortsætter netværket med pooling.

Pooling bevarer vigtig information, mens mindre relevante detaljer kasseres og skaber rumlige hierarkier af features. Den mest almindelige form for pooling er "max pooling", som beholder den største værdi inden for et pooling-vindue. Hvis vi tager et pooling-vindue af størrelse 2 gange 2 og anvender det på den aktiverede feature map, vi producerede sammen i det foregående trin, vil resultatet se således ud:

<img src="./images/Pooling-Main.png" style="width:800px; height:1000px;"/>

### Fuld forbundne lag

Efter pooling bliver feature maps foldet ud eller *fladgjort*, så de fremstår som vektorer af tal (ligesom i tabulerede data) og overføres til en række fuldt forbundne lag, efterfulgt af outputlaget. Det fuldt forbundne lag ligner det, vi havde i tilfældet med Iris-dataene, det består af lineære neuroner og en aktiveringsfunktion:

<img src="./images/Fully connected.png" style="width:800px; height:400px;"/>

## Implementering af CNN i PyTorch

Lad os bygge et simpelt CNN-netværk til klassificering af farvebilleder ved hjælp af PyTorch.

Vi vil antage, at inputbilledet kun har tre kanaler (RGB). Det første konvolutionelle lag vil tage et tre-kanals billede og producere fire forskellige feature maps, som om man anvendte fire forskellige filtre på det samme billede. Derefter vil vi anvende en aktiveringsfunktion og samle features ved hjælp af et 2x2 max pooling-lag.

Fordi vi også vil tilføje padding, vil det konvolutionelle lag producere et feature map af samme størrelse som de originale billeder. Men efter pooling-laget vil det blive halvt så stort. Hvis de originale billeder har en størrelse på 256x256 pixels, vil vi efter pooling få 4 feature maps med 128x128 pixels hver.

Derefter tilføjer vi det andet konvolutionelle lag, som vil anvende sine filtre på de feature maps, der er produceret og samlet i de foregående trin. Det vil tage de fire feature maps fra det foregående lag og producere otte nye feature maps som resultat.

Feature maps'ene vil også blive samlet, så efter pooling vil vi få 8 feature maps med 64x64 pixels hver (for et billede på 256x256), hvilket giver en vektor med 32.768 feature-værdier, som skal flades og sendes til de fuldt forbundne lag.

Her er den fulde kode til vores netværk:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class ImageClassifier(nn.Module):
    def __init__(self, width, height):
        super().__init__()

        # first convolutional layer
        # - takes 3 channel image (RGB) and produces 4 channels (maps) with features
        # - it uses kernel of size 3x3 and adds a pad of 1 pixel around to keep the same size
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, stride=1, padding=1)        # convolutional layer 1
        # max pooling layer
        # - has size of 2x2 so it reduces size of feature map twice
        self.pool = nn.MaxPool2d(2, 2)
        # second convolutional layer
        # - takes 4 channels (feature maps) and produce 8 channels with features
        # - it uses kernel of size 3x3 and adds a pad of 1 pixel around to keep the same size
        self.conv2 = nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3, stride=1, padding=1)         # convolutional layer 2

        # set of fully connected layers, number of inputs in this case depends on width
        # and height of the original image (both will be reduced by 4)
        self.fc1 = nn.Linear(8 * width // 4 * height // 4, 1024)
        self.fc2 = nn.Linear(1024, 256)
        # the output layer in this case has two outputs — one for each class
        # the classification decision will be made by taking the biggest of the
        # two output values
        self.fc3 = nn.Linear(256, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))    # send input to 1st convolutional layer and ReLU
        x = self.pool(x)             # pool the feature maps from previous layer
        x = F.relu(self.conv2(x))    # send pooled features to 2nd convolutional layer and ReLU
        x = self.pool(x)             # pool the feature maps again

        x = torch.flatten(x, 1)      # flatten the outputs from the previous layer
        x = F.relu(self.fc1(x))      # send flattened features to the 1st linear layer + ReLU
        x = F.relu(self.fc2(x))      # send output of the 1st layer to the 2nd linear layer + ReLU
        y_hat = self.fc3(x)          # send output of the 2nd layer to the output layer
        return y_hat

Lad os initialisere modellen til billeder af størrelsen 256x256 pixels og se på oversigten:

In [ ]:
from torchinfo import summary

model = ImageClassifier(256, 256)
summary(model)

For eksempel bør den første konvolutionslag have 12 kerner, hver på 3x3 (én kerne til hver input og hver output). Plus fire bias, hvilket giver: $3 \times 4 \times 3 \times 3 + 4 = 112$ parametre.

For den anden har vi: $4 \times 8 \times 3 \times 3 + 8 = 296$ parametre.

Og så videre, i alt har modellen mere end 30 millioner parametre at træne!

## Klassifikation af ansigter

Nu skal vi lære, hvordan vi kan bruge CNN-netværket, vi lige har defineret, til rigtige datasæt.

### Indlæs billeder som datasæt

Først og fremmest er det værd at nævne, at PyTorch har et særligt ekstra bibliotek, `torchvision`, som hjælper med at indlæse billeder som datasæt, tildele dem etiketter osv. Dette bibliotek giver også en modul `transforms`, der kan anvende forskellige transformationer, som vi gjorde i den første klasse: beskæring, ændring af størrelse osv.

For CNN er det vigtigt, at:

1. Alle billeder har samme størrelse (samme antal pixels).
2. Alle billeder har samme farvemodel eller gråtoneformat.
3. Pixels har intensitet mellem 0 og 1.
4. Alle billeder er PyTorch-tensorer.

Alt dette kan opnås ved at definere en sekvens af transformationsmetoder fra modul `torchvision.transforms`, som Torch automatisk vil anvende på hvert billede.

Lad os se på følgende kode:

In [ ]:
from torchvision import datasets, transforms

# define same size for all images
img_width = 256
img_height = 256

# define path to folder with images for each subset
image_path = "dataset"

# transformation sequence
transform = transforms.Compose([
    transforms.Resize([img_width, img_height]), # resize so each image has the same size
    transforms.ToTensor()                       # convert to PyTorch tensor
])

# create a dataset based on the image folder structure and defined transformation
dataset = datasets.ImageFolder(root=image_path, transform=transform)
dataset

Først indlæser vi to moduler fra `torchvision`. Modulet `dataset` indeholder metoder, der kan indlæse billeder fra disken, tildele labels, transformere dem til Torch tensors og kombinere dem til et dataset. Modulet `transform`, som allerede nævnt, indeholder metoder til transformation af billeder.

Derefter definerer vi bredden og højden af de målrettede billeder i pixels. Alle billeder vil blive ændret til denne størrelse. Derefter definerer vi placeringen af billederne på disken (sti til mappen).

Hvis du åbner mappen `dataset`, kan du se, at der inde i denne mappe er to andre mapper. Mappen `glasses` indeholder billeder af ansigter med briller, og mappen `noglasses` indeholder ansigtsbilleder uden briller. Tjek flere billeder fra hver mappe.

Metoden `ImageFolder`, som vi bruger til at indlæse billederne, "kender" til dette. Så den vil tildele alle billeder fra den første undermappe til klasselabelen `"glasses"` og alle billeder fra den anden undermappe til klasselabelen `"no glasses"`.

Lad os undersøge datasettet lidt mere:

In [ ]:
# shows list of classes
dataset.classes

In [ ]:
# numeric labels for each class
dataset.class_to_idx

In [ ]:
# show number of elements in the dataset
len(dataset)

In [ ]:
# list of first 10 images
dataset.samples[0:10]

Som du kan se, inde i `dataset` har vi ikke tensorer med billede pixels, men bare en fuld sti til hvert billede og en numerisk label, som er forbundet med tekstklasselabelen. Billederne vil blive indlæst under trænings- og forudsigelsesprocesserne. Denne tilgang hjælper med at spare din computers hukommelse.

Lad os vise nogle af billederne ved hjælp af PIL-biblioteket, som vi lærte om i den første klasse:

In [ ]:
img_indices = range(0, 4000, 200)
list(img_indices)
list(range(len(img_indices)))

img_indices[2]

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# take every 200th of the first 4000 images
img_indices = range(0, 4000, 200)

plt.figure(figsize = (10, 8))

for i in range(len(img_indices)):
    path, class_ind = dataset.samples[img_indices[i]]
    img = Image.open(path)

    plt.subplot(4, 5, i + 1)
    plt.imshow(np.array(img))
    plt.axis('off')
    plt.title(dataset.classes[class_ind])


Endelig, lad os opdele hele datasættet til trænings-, validerings- og testsæt. Denne gang vil vi gøre det tilfældigt ved at bruge funktionen `random_split` fra Torch.

Den ideelle opdeling vil være at tage 70% til træning, 20% til validering og 10% til test. Men vores datasæt er stort (4000+ billeder), og selv at bruge 70% af det til træning vil gøre processen meget lang, indtil vi kører den på en kraftfuld computer med GPU.

For at spare tid vil vi kun tage 20% (800+) af billederne til træning, 10% til validering og 10% til test. Men fordi `random_split` kræver, at alle procenter skal summe til 100%, vil vi oprette det fjerde subset, `rest_set`, som vi simpelthen ikke vil bruge.

Senere, når du lærer alt indholdet, skal du forsøge at øge træningssættet og se, hvordan det påvirker modelkvaliteten. Generelt vil jo flere data du har, desto mere effektiv vil træningsprocessen være.

In [ ]:
nall = len(dataset)
ntrain = int(nall * 0.20)
ntrain

In [ ]:
from torch.utils.data import random_split

nall = len(dataset)
ntrain = int(nall * 0.20)
nval = int(nall * 0.10)
ntest = int(nall * 0.10)
nrest = nall - ntrain - nval - ntest


# we need a seed here as well because of random splits
torch.manual_seed(12)
[train_set, val_set, test_set, rest_set] = random_split(dataset, (ntrain, nval, ntest, nrest))


### Træning af CNN-modellen

Træningsprocessen for CNN-modellen er næsten identisk med den, vi brugte i den forrige klasse. Vi vil dog introducere to forskelle.

Første vigtige forskel er, at vi ikke vil sende alle billeder på én gang til træningsprocessen. Det ville kræve en masse hukommelse og computerkraft. I stedet vil vi gøre det i små portioner - *batches*.

Så vi vil opdele alle trænings- og valideringssæt i batches og lave en løkke, så den tager billeder fra det første batch, træner modellen med dette batch. Derefter tager billeder fra det andet batch, træner modellen med det andet batch, og så videre.

Denne træningsmetode er mere effektiv og også lidt hurtigere. Hastighed er vigtig, fordi i dette tilfælde er vores model meget stor, og billeder indeholder mange pixels, så træningsprocessen vil være meget langsommere end i tilfælde af Iris-klassifikation.

For at bruge batches har PyTorch en speciel klasse, som hedder `DataLoader`. Lad os oprette indlæsere for trænings- og valideringssættene.  

In [ ]:
# import data loader class
from torch.utils.data import DataLoader

# define how many images will be in one batch
batch_size = 10

# create data loaders with this batch size
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

Som du kan se, er der en speciel parameter, `shuffle`, som er sat til `True` for træningssættets indlæser. Dette betyder, at billeder vil blive sorteret tilfældigt i batches. Dette er meget vigtigt, da det hjælper med at undgå situationer, hvor f.eks. alle billeder i et batch indeholder ansigter uden briller, så modellen ikke kan lære noget af et sådan batch.

Batch-størrelsen er en anden (sammen med læringsraten) vigtig indstilling, som kan påvirke træningskvaliteten, så det giver mening at variere den lidt, hvis kvaliteten af den trænede model ikke er tilfredsstillende.

Disse indstillinger, som ikke kan optimeres automatisk, og det er dit ansvar at finde de gode, kaldes *hyperparametre*. Så enhver model har *parametre* (såsom vægte og bias for neuronerne), som estimeres automatisk under læringsprocessen, og *hyperparametre* (såsom læringsrate, batch-størrelse, antal epocher osv.), som skal optimeres manuelt af en data scientist.

Her er en kode, der implementerer batch-læring (vi gør det også som en funktion som i den tidligere klasse). Vi bruger den samme optimizer og samme tabsfunktion som i det endelige eksempel med Iris. Processen vil tage op til 10-15 minutter, så det kan være en god idé at starte den, sikre dig, at den fungerer, og tage en pause:

In [ ]:
def train_model(model, train_loader, val_loader, nepochs = 100, lr = 0.001):
    """ trains CNN model with provided data """

    # define a loss function
    loss_function = nn.CrossEntropyLoss()

    # define optimizer which will compute gradients — do the learning.
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    # prepare arrays for losses
    train_losses = np.zeros(nepochs)
    val_losses = np.zeros(nepochs)

    # training loop
    for epoch in range(nepochs):  # Number of training epochs

        # train
        model.train()
        train_loss = 0
        for batch_data in train_loader:
            inputs, labels = batch_data
            optimizer.zero_grad()
            labels_predicted = model(inputs)
            loss = loss_function(labels_predicted, labels)
            loss.backward()
            optimizer.step()
            train_loss = train_loss + loss.item()
        train_losses[epoch] = train_loss / len(train_loader)

        # validate
        val_loss = 0
        model.eval()
        for batch_data in val_loader:
            inputs, labels = batch_data
            labels_predicted = model(inputs)
            loss = loss_function(labels_predicted, labels)
            val_loss = val_loss + loss.item()
        val_losses[epoch] = val_loss / len(val_loader)

        # show how big the losses are at this epoch
        print(f'Epoch {epoch}, train loss: {train_losses[epoch]:.4f} - validation loss {val_losses[epoch]:.4f}')

    return train_losses, val_losses


In [ ]:
# seed the random numbers generator to get reproducible results
torch.manual_seed(12)

# initialize the model
model = ImageClassifier(img_width, img_height)

# train it for 40 epochs and learning rate of 0.01
train_losses, val_losses = train_model(model, train_loader, val_loader, nepochs = 40, lr = 0.01)

Lad os se på tabets værdier:

In [ ]:
# show plot with losses
plt.plot(train_losses, label = "train")
plt.plot(val_losses, label = "val")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Loss")

Her har vi flere problemer. Først og fremmest kan du se en underlig opførsel af valideringstab, den hopper op og ned. Måske skal vi bruge en mindre læringsrate, det vil vi finde ud af senere.

Det andet problem er, at fra omkring den 20. epoch begynder valideringstabett at stige langsomt, så den endelige model er ikke den mest optimale. Lad os tale om, hvordan man løser dette problem lidt senere, men lad os nu tjekke præstationen af modellen på testsættet.

For at gøre dette skal vi skrive en ny funktion til at lave forudsigelser. Som du husker, brugte vi denne gang datasæt baseret på billedmapper. Disse typer datasæt indeholder stier til billederne og tilhørende etiketter, og derefter bruger vi dataindlæsere til at indlæse billederne fra disken, forbehandle dem, opdele dem i batches og fodre modellen med batches.

For at lave forudsigelser har vi ikke brug for batches, men at bruge dataindlæseren er stadig praktisk, da den automatiserer mange ting. Hvad vi kan gøre er at oprette en indlæser med et enkelt batch, så alle billederne vil være i det batch, og bruge det til at foretage forudsigelser.

Her er den:

In [ ]:
def predict(model, dataset):
    """ get ANN model and tensor with predictors and returns predicted class label indices """
    model.eval()
    data_loader = DataLoader(dataset, batch_size = len(dataset))
    for inputs, labels in data_loader:
        output = model.forward(inputs)
        _, predicted_labels = torch.max(output, 1)

    return labels, predicted_labels

Som du kan se, returnerer denne funktion både labels og forudsagte labels, så vi nemt kan genbruge de andre funktioner, vi oprettede i den sidste klasse, for at kontrollere modelpræstationen - beregning af kryds-tabellen og visualisering af denne tabel som et varmekort.

Lad os først lave forudsigelser:

In [ ]:
predicted_labels, labels = predict(model, test_set)

Og visualiser præstationen.

In [ ]:
ct = table(labels, predicted_labels)
ct_heatmap(ct, dataset.classes)

Ikke dårligt overhovedet, vel? Selvfølgelig vil resultatet variere, hvis du kommenterer `manual_seed()`-linjen ud og kører den flere gange, fordi i dette tilfælde vil vægtene blive initialiseret tilfældigt, og hver gang du kører træningsprocessen, vil du få en forskellig præstation.

Nu har du opnået noget nyt — du har skabt og trænet en CNN-model, der automatisk kan genkende mennesker med briller. Tilsvarende modeller kan udføre mere nyttigt arbejde, f.eks. opdage, om en bilist bruger mobiltelefon under kørsel (du har nok hørt, at dette er ulovligt) eller sortere forskellige objekter (f.eks. affald, grøntsager eller lignende).

Der er stadig et par ting at lære, men nu er det tid til en øvelse:

### Øvelse

I den tidligere lektion lærte du, hvordan man gemmer en statusdictionary for en model på et hvilket som helst tidspunkt til en variabel (ved at tage en dyb kopi af ordbogen). Modificer funktionen `train_model()` i kodeblokken nedenfor, så den altid resulterer i en model med den laveste valideringsfejl.

For eksempel, hvis du kører en model i 100 epocher, og den laveste valideringstab blev opnået ved epoch 67, vil funktionen gemme status for denne model, og ved slutningen af træningsløkken vil den indlæse denne status i den aktuelle model.

In [ ]:
from copy import deepcopy

def train_model(model, train_loader, val_loader, nepochs = 100, lr = 0.001):
    """ trains CNN model with provided data """

    # define a loss function
    loss_function = nn.CrossEntropyLoss()

    # define optimizer which will compute gradients — do the learning.
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    # prepare arrays for losses
    train_losses = np.zeros(nepochs)
    val_losses = np.zeros(nepochs)

    # HINT: here you need to initialize two variables
    # - one will keep the best validation loss value
    # - second one will keep the state dictionary of the model you got this loss for
    best_model = None
    best_loss = 99999999999.0

    # training loop
    for epoch in range(nepochs):  # Number of training epochs

        # train
        model.train()
        train_loss = 0
        for batch_data in train_loader:
            inputs, labels = batch_data
            optimizer.zero_grad()
            labels_predicted = model(inputs)
            loss = loss_function(labels_predicted, labels)
            loss.backward()
            optimizer.step()
            train_loss = train_loss + loss.item()
        train_losses[epoch] = train_loss / len(train_loader)

        # validate
        val_loss = 0
        model.eval()
        for batch_data in val_loader:
            inputs, labels = batch_data
            labels_predicted = model(inputs)
            loss = loss_function(labels_predicted, labels)
            val_loss = val_loss + loss.item()
        val_losses[epoch] = val_loss / len(val_loader)

        # show how big the losses are at this epoch
        print(f'Epoch {epoch}, train loss: {train_losses[epoch]:.4f} - validation loss {val_losses[epoch]:.4f}')

        # HINT:
        # here you need to add a condition which will compare current model with the
        # best one you got so far. If the current model is better, you save its state as
        # the new best. You also need to save the best loss value — this is the way to
        # see if the next model will be even better
        if val_losses[epoch] < best_loss:
            best_model = deepcopy(model.state_dict())
            best_loss = val_losses[epoch]

    # HINT:
    # here you need to load the state of the best model from the loop
    # to your model object
    model.load_state_dict(best_model)

    return train_losses, val_losses

Løsningen:

In [ ]:
from copy import deepcopy

def train_model(model, train_loader, val_loader, nepochs = 100, lr = 0.001):
    """ trains CNN model with provided data """

    # define a loss function
    loss_function = nn.CrossEntropyLoss()

    # define optimizer which will compute gradients — do the learning.
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    # prepare arrays for losses
    train_losses = np.zeros(nepochs)
    val_losses = np.zeros(nepochs)

    # initialize two variables which will keep the best validation loss and the best model state
    best_loss = 999999999999
    best_model_state = None

    # training loop
    for epoch in range(nepochs):  # Number of training epochs

        # train
        model.train()
        train_loss = 0
        for batch_data in train_loader:
            inputs, labels = batch_data
            optimizer.zero_grad()
            labels_predicted = model(inputs)
            loss = loss_function(labels_predicted, labels)
            loss.backward()
            optimizer.step()
            train_loss = train_loss + loss.item()
        train_losses[epoch] = train_loss / len(train_loader)

        # validate
        val_loss = 0
        model.eval()
        for batch_data in val_loader:
            inputs, labels = batch_data
            labels_predicted = model(inputs)
            loss = loss_function(labels_predicted, labels)
            val_loss = val_loss + loss.item()
        val_losses[epoch] = val_loss / len(val_loader)

        # show how big the losses are at this epoch
        print(f'Epoch {epoch}, train loss: {train_losses[epoch]:.4f} - validation loss {val_losses[epoch]:.4f}')

        # check if validation loss is better to what is known so far
        # if so save the model state
        if val_losses[epoch] < best_loss:
            best_loss = val_losses[epoch]
            best_model_state = deepcopy(model.state_dict())

    # load the state from the best model
    model.load_state_dict(best_model_state)
    return train_losses, val_losses

Efter du har skrevet din funktion, kan du teste den ved at bruge koden nedenfor:

In [ ]:
# seed the random numbers generator to get reproducible results
torch.manual_seed(12)

# initialize the model
model = ImageClassifier(img_width, img_height)

# train it for 40 epochs and learning rate of 0.01
train_losses, val_losses = train_model(model, train_loader, val_loader, nepochs = 40, lr = 0.01)

In [ ]:

# compute loss of the final model on validation set
loss_function = nn.CrossEntropyLoss()
val_loss = 0
model.eval()
for batch_data in val_loader:
    inputs, labels = batch_data
    labels_predicted = model(inputs)
    loss = loss_function(labels_predicted, labels)
    val_loss = val_loss + loss.item()
val_loss = val_loss / len(val_loader)

val_loss

Hvis du ser, at tabet, vi fik i denne test, er mindre end tabet vist for den sidste epoke, da vi trænede modellen, virker det.

Lad os visualisere dette ved at vise et diagram med tab fra træningsprocessen og tabet af den endelige model som en vandret linje. Hvis linjen rører ved valideringstabkurven i det globale minimum, fungerer din funktion korrekt.

In [ ]:
# show plot with losses
plt.plot(train_losses, label = "train")
plt.plot(val_losses, label = "val")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Loss")

# show the recently computed loss as horizontal line
plt.plot(plt.xlim(), [val_loss, val_loss], color = "black", linestyle = "--")

### At lave forudsigelser for nye billeder

Hvad nu hvis vi vil at lave en forudsigelse for et nyt billede? Bare et enkelt billede, hvis rigtige label vi ikke kender, så vi kan ikke bruge dataindlæseren i dette tilfælde. Lad os tage billednummer 2000 fra det originale datasæt til dette formål:

In [ ]:
# dataset items contain two elements - path and label, so we take the first one
new_image_path = dataset.imgs[2000][0]
new_image_path

In [ ]:
from PIL import Image
img = Image.open(new_image_path)
display(img)

Så personen bærer briller, lad os se, hvordan man laver forudsigelser uden dataloader:

In [ ]:
# manually apply all transformations to the new image
img_transformed = transform(img)

# reshape image tensor because model does not work with single images
# it expects tensor of images. So if we need to feed model the image
# we make it a tensor with one image
img_transformed = img_transformed.reshape(1, 3, img_width, img_height)

# apply the model
output = model.forward(img_transformed)

# compute label
_, labels_predicted = torch.max(output, 1)

# show all outcomes
(output, labels_predicted, dataset.classes[labels_predicted])

### Anvend netværket på kamerabillede

Nu vil vi lære, hvordan vi kan bruge modellen til at lave forudsigelser i realtid ved at tage billeder med vores frontkameraer. Kør den næste kode for at få billedet af dit ansigt eller et ansigt af din ven.

In [ ]:
# connect to camera
import cv2
camera = cv2.VideoCapture(0)
camera.isOpened()

In [ ]:
# take a picture (repeat if necessary to get better result)
ret, frame = camera.read()
plt.imshow(frame)

In [ ]:
# close connection to the camera
camera.release()

Nu, brug koordinataksene til at definere en beskæringsrektangel. Sørg for, at det endelige ansigt ligger midt i det beskårne billede:

In [ ]:
# left top right bottom
crop_rect = [250, 70, 474, 294]
img = Image.fromarray(frame)
img_cropped = img.crop(crop_rect)
print(img_cropped.size)
display(img_cropped)

Anvend modellen og se forudsigelse.

In [ ]:
img_transformed = transform(img_cropped)
img_transformed = img_transformed.reshape(1, 3, 256, 256)
output = model.forward(img_transformed)

# compute label
_, labels_predicted = torch.max(output, 1)

# show all outcomes
(output, labels_predicted, dataset.classes[labels_predicted])

## Sådan udfører du beregninger på GPU'en.

Du har muligvis bemærket, at i modsætning til eksperimenter med Iris-data kræver træning og brug af model til billeder meget længere tid. Denne proces kan fremskyndes, hvis du har en NVIDIA GPU på din computer (selv den enkleste, som RTX 1060).

For at gøre dette skal du sende din model og dine data til GPU-enheden ved hjælp af en speciel metode. Herunder finder du en kode, der implementerer denne tilgang. Koden er alsidig, hvilket betyder, at hvis du har en GPU, vil den bruge den, men hvis ikke, kan du køre din kode på en CPU uden at ændre noget.

Først og fremmest lad os lære at detektere den kompatible GPU-enhed. I forhold til Torch kaldes en sådan enhed [CUDA](https://en.wikipedia.org/wiki/CUDA) (`cuda`). Det er navnet på den tilsvarende computerbibliotek, der lader os bruge af GPU-enheder til beregninger. Her er hvordan du kan tjekke om du har en:

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.device("cuda")

Og denne kode vil automatisk definere den tilgængelige enhed og gemme den i en separat variabel. Hvis du har CUDA, vil den vælge det, hvis ikke vil den i stedet vælge din CPU:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

Nu skal vi omskrive træningsloopen for at bruge den registrerede enhed til træning. Vi genbruger dataindlæsere og modelklassen, du har oprettet før i denne klasse, så for at få det til at virke, skal du køre de tidligere kodeceller.

Koden holdes så simpel som muligt uden valideringsloop, blot for at give dig en idé. Som du kan se, er der kun to ændringer, en linje hvor vi sender modellen til enheden (`model.to(device)`). Og det andet linje, hvor vi gør det samme med inputs og labels (`inputs.to(device)` og `labels.to(device)`).

In [ ]:
# initialize a model
model = ImageClassifier(img_width, img_height)

# define number of epochs and learning rate
nepochs = 3
lr = 0.01

# define a loss function
loss_function = nn.CrossEntropyLoss()

# define optimizer which will compute gradients — do the learning.
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

# NEW: send the model to device you defined earlier
model.to(device)

# training loop
for epoch in range(nepochs):  # Number of training epochs

    # train
    model.train()
    train_loss = 0
    for batch_data in train_loader:
        inputs, labels = batch_data

        # NEW: send batch data to the device as well
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        labels_predicted = model(inputs)
        loss = loss_function(labels_predicted, labels)
        loss.backward()
        optimizer.step()
        train_loss = train_loss + loss.item()

    train_losses = train_loss / len(train_loader)
    print(f'Epoch {epoch}, train loss: {train_loss:.4f}')


Hvis du træner din model på en GPU og vil lave forudsigelser, skal den nye data (test sæt, validering sæt eller nyt billede) også indlæses på GPU'en først. Alternativt kan du aflaste din model fra GPU/CUDA til CPU ved at køre: `model.to(torch.device("cpu"))` efter træning.

## Overførselslæring

Som du måske har bemærket, tager træningsprocessen, selv for et relativt simpelt netværk som vi har oprettet på et relativt lille datasæt, lang tid. Brug af en GPU/CUDA kan delvist løse dette problem. Gode modeller med høj nøjagtighed er langt mere komplicerede og er blevet trænet på langt større datasæt. Kan vi udnytte fordelene ved modellen, som nogen andre har trænet?

Ja, det kan vi, og dette er præcis hvad *overførselslæring* gør. Ideen er, at du tager en model, der allerede er blevet trænet på et meget bredt udvalg af billeder og billedklasser. Derefter finjusterer du denne model, så den fungerer på dit specifikke datasæt.

Fordi vægtene i denne model allerede er sat, behøver du ikke at starte læringsprocessen fra bunden, og dette sparer mange ressourcer.

Torch har allerede flere forbehandlede modeller, herunder berømte som [AlexNet](https://en.wikipedia.org/wiki/AlexNet), [VGG](https://www.kaggle.com/code/blurredmachine/vggnet-16-architecture-a-complete-guide) og mange andre. Modeller til billedeklassificering findes i `torchvision.models`. Lad os indlæse en af disse ([residuelt netværk](https://en.wikipedia.org/wiki/Residual_neural_network) med 18 lag, der er også en med 50):

In [ ]:
from torchvision import models
from torchinfo import summary

model = models.resnet18()
summary(model)

Som du kan se, har modellen færre parametre end vores, men arkitekturen er meget mere kompleks, hvilket gør den mere effektiv og alsidig (outputtet er faktisk forkortet, så du ikke ser den fulde struktur).

For at bruge netværket skal vi vide, hvordan man transformerer (forbehandler) billederne, og hvad billedstørrelsen skal være. Heldigvis kan vi få den allerede forberedte række af transformationer, der er tilsluttet til vægtene i modellen.

Her er hvordan man får den.

In [ ]:
weights = models.ResNet18_Weights.DEFAULT
transform = weights.transforms()
transform

Fordi transformationsstakken er anderledes end den, vi brugte tidligere, skal vi genskabe datasættet, delmængderne og dataloadere. Vi gentager bare koden med det nye transformationsobjekt:

In [ ]:
# create a dataset based on the image folder structure and defined transformation
dataset = datasets.ImageFolder(root=image_path, transform=transform)
dataset

In [ ]:
from torch.utils.data import random_split

nall = len(dataset)
ntrain = int(nall * 0.20)
nval = int(nall * 0.10)
ntest = int(nall * 0.10)
nrest = nall - ntrain - nval - ntest


torch.manual_seed(12)

[train_set, val_set, test_set, rest_set] = random_split(dataset, (ntrain, nval, ntest, nrest))

Nu opretter vi dataindlæserne:

In [ ]:
# import data loader class
from torch.utils.data import DataLoader

# define how many images will be in one batch
batch_size = 10

# create data loaders with this batch size
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

Før træning skal vi faktisk modificere modellen. Den oprindelige model kan klassificere billeder blandt 1000 klasser, så den har 1000 outputs. Du kan se dette, hvis du tjekker outputlaget, som har navnet `fc` i denne model:

In [ ]:
model.fc

Så vi kan justere det ved at oprette et nyt lineært lag med lige så mange input som det originale og kun to outputs. Vi kan erstatte en del af ResNet18-modellen. Sådan gør du det:

In [ ]:
# replace the output layer in the ResNet18 model by a new one with 2 outputs
in_features = model.fc. in_features
model.fc = nn.Linear(in_features, 2)

Nu er vi klar til at træne, eller rettere finjustere, modellen. Vi vil starte med blot 10 epoker. Hvis du har oprettet den smarte `train_model()` metode i en af øvelserne, vil den ende med den mest optimale model.

In [ ]:
train_losses, val_losses = train_model(model, train_loader, val_loader, nepochs=10, lr = 0.01)

Som du kan se, allerede efter 1-2 epoker bliver tabet meget lille. Lad os tjekke præstationen:

In [ ]:
labels, predicted_labels = predict(model, test_set)
ct = table(labels, predicted_labels)
ct_heatmap(ct, dataset.classes)

Perfekt resultat og meget hurtigere end at træne CNN-modellen fra bunden!

## Hvad skal jeg gøre næste?

Hvis du vil at fortsætte og udvikle dine færdigheder yderligere, er her nogle nyttige links.

For at lære Python på en mere systematisk måde, kan du kigge på følgende materialer:

* [Python for børn](https://www.geeksforgeeks.org/python-for-kids/)
* [Google Python class](https://developers.google.com/edu/python)
* [Python tutorial på geeksforgeeks.org](https://www.geeksforgeeks.org/python-programming-language-tutorial/)
* [Videnskabelige Python-forelæsninger](https://lectures.scientific-python.org/)

Der er selvfølgelig mange flere, og Python er sandsynligvis det mest populære programmeringssprog i dag.

Hvad angår datavidenskab, maskinlæring eller kunstig intelligens, er der også hundredevis af gode kurser. Vi anbefaler en [gratis online kursus](https://course.fast.ai) fra FastAI, som dækker alle aspekter af moderne ML/AI, herunder mere sofistikerede emner som Stable Diffusion. Det er også baseret på Jupyter notebooks, så du vil føle dig komfortabel fra starten.

God læring!